# A Markdown cell
- You can do all the fun stuff markdown has to offer

In [ ]:
# # Imports
import pandas as pd
import folium

In [ ]:
my_map = folium.Map(location=[50, -122], 
                    zoom_start=4)

folium.TileLayer('Stamen Terrain').add_to(my_map)
my_map.add_child(folium.map.LayerControl())

my_map # display map